In [96]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple
from scipy.spatial.distance import cosine as cosine_distance
from collections import defaultdict


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
with open("queries2.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]


In [98]:
queries = queries[:220]

In [99]:
for i,q in enumerate(queries):
    a = q.count("arg1:") == q.count("arg2:") == 1
    if not a:
        print(i,q)

In [100]:
queries2results = defaultdict(list)

In [102]:
dataset_name = "covid19"
num_results = 100
query_type = "syntactic"

for q_and_id in tqdm.tqdm(queries, total = len(queries)):
    q, id = q_and_id.split("\t")
    id = int(id)
    df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
    queries2results[id].append(df)

  1%|          | 2/220 [00:24<44:02, 12.12s/it]


HTTPError: HTTP Error 503: Service Temporarily Unavailable

In [ ]:
queries2results[1][0].head()

In [ ]:
with open("queries2results.pickle", "wb") as f:
     pickle.dump(queries2results, f)

In [8]:
with open("queries2results.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [63]:
queries2results.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0])

In [65]:
queries2results_new = dict()
for k,dfs in queries2results.items():
    if k == 0: continue
    queries2results_new[k] = pd.concat(dfs, axis=0)
    
queries2results = queries2results_new

/home/shauli/miniconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [66]:
ids = set([int(l.split("\t")[-1]) for l in queries])

In [71]:
data = []
for id in ids:
    results = queries2results[id]
    sents, arg1_first, arg2_first = results["sentence_text"].tolist(), results["arg1_first_index"].tolist(), results["arg2_first_index"].tolist()
    arg1_last, arg2_last = results["arg1_last_index"].tolist(), results["arg2_last_index"].tolist()
    
    sents_with_args = []
    for s,arg1_ind,arg2_ind, arg1_ind_last, arg2_ind_last in zip(sents,arg1_first,arg2_first, arg1_last, arg2_last):
        s_lst = s.split(" ")
        if arg1_ind > arg2_ind:
            arg1_ind, arg2_ind = arg2_ind, arg1_ind
            arg1_ind_last, arg2_ind_last = arg2_ind_last, arg1_ind_last
            arg1_str, arg2_str = "ARG2:", "ARG1:"
        else:
            arg1_str, arg2_str = "ARG1:", "ARG2:"
        s_with_args = s_lst[:arg1_ind] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
        s_with_args = " ".join(s_with_args)
        sents_with_args.append({"sent": s_with_args, "start_1": arg1_ind, "start_2": arg2_ind, "end_1": arg1_ind_last,
                               "end_2": arg2_ind_last})
    
    max_number_of_pairs = 450
    pairs = list(itertools.product(sents_with_args, repeat=2))
    print(id, len(pairs))
    random.shuffle(pairs)
    for pair in pairs[:max_number_of_pairs]:
        data.append({"first": pair[0]["sent"], "second": pair[1]["sent"], "query": q, "first_arg1": (pair[0]["start_1"], pair[0]["end_1"]),
                    "first_arg2": (pair[0]["start_2"], pair[0]["end_2"]), "second_arg1": (pair[1]["start_1"], pair[1]["end_1"]),
                   "second_arg2": (pair[1]["start_2"], pair[1]["end_2"])})

1 136161
2 54289
3 90000
4 234256
5 370881
6 93636
7 63504
8 4
9 130321
10 158404
11 111556
12 7921
13 50625
14 134689
15 41616


In [72]:
len(data)

6304

In [73]:
random.shuffle(data)
data[0]

with open("data.pickle", "wb") as f:
    pickle.dump(data, f)

In [74]:

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [75]:
import json

with open("data.txt", "w") as f:
    for d in data:
        first, second = d["first"], d["second"]
        first_arg1 = d["first_arg1"]
        first_arg2 = d["first_arg2"]
        second_arg1 = d["second_arg1"]
        second_arg2 = d["second_arg2"]
        
        elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
        keys = ["first", "second", "first_arg1", "first_arg2"]
        
        f.write(json.dumps(d) + "\n")
        #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")

In [76]:
random.shuffle(data)
data[9]

{'first': 'The ultimate treatment would be liver transplant , but ARG1:this is contraindicated in ARG2:patient with such short sobriety .',
 'second': 'This could leave us with only 1 antiviral class of drugs ( ie , neuraminidase inhibitors ) and no treatment option for infants , because ARG1:oseltamivir is contraindicated in ARG2:children less than 1 year of age based on a study , done by the manufacturer , showing increased central nervous system drug concentrations in a juvenile animal model .',
 'query': '<>arg1:SCD $[l]polymorphism is a $risk factor for <>arg2:something',
 'first_arg1': (9, 9),
 'first_arg2': (13, 13),
 'second_arg1': (25, 25),
 'second_arg2': (29, 29)}

In [77]:
global count
count = 0
global total
total = 0

In [78]:
class BertModel(pl.LightningModule):

    def __init__(self, train_dataset: Dataset, dev_dataset: Dataset, batch_size, device: str, mode: str = "eval"):
        
        super().__init__()
        
        self.device = device
        config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased', output_hidden_states=True)
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        self.model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config)    
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.linear_arg1_1 = torch.nn.Linear(768, 128)
        self.linear_arg2_1 = torch.nn.Linear(768, 128)
        self.linear_arg1_2 = self.linear_arg1_1 #torch.nn.Linear(768, 256)
        self.linear_arg2_2 = self.linear_arg2_1 #torch.nn.Linear(768, 256)
        
        if mode == "eval":
            
            self.model.eval()
        else:
            self.model.train()
        
        for p in self.model.parameters():
            p.requires_grad = False
        for p in self.model.encoder.layer[-1].parameters():
            p.requires_grad = True
        for p in self.model.encoder.layer[-2].parameters():
            p.requires_grad = False       

        for p in self.model.embeddings.parameters():
            p.requires_grad = False
            
        self.linear_arg1_1.requires_grad = True
        self.linear_arg2_1.requires_grad = True
        self.linear_arg1_2.requires_grad = True
        self.linear_arg2_2.requires_grad = True
        
    
        self.train_gen = torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_dataset, batch_size=batch_size, drop_last=False, shuffle=False)
        self.acc = None

        
    def tokenize(self, original_sentence: List[str]) -> Tuple[List[str], Dict[int, int]]:

        """
        Parameters
        ----------
        Returns
        -------
        bert_tokens: The sentence, tokenized by BERT tokenizer.
        orig_to_tok_map: An output dictionary consisting of a mapping (alignment) between indices in the original tokenized sentence, and indices in the sentence tokenized by the BERT tokenizer. See https://github.com/google-research/bert
        """

        bert_tokens = ["[CLS]"]
        orig_to_tok_map = {}
        tok_to_orig_map = {}
        has_subwords = False
        is_subword = []

        for i, w in enumerate(original_sentence):
            tokenized_w = self.tokenizer.tokenize(w)
            has_subwords = len(tokenized_w) > 1
            is_subword.append(has_subwords)
            bert_tokens.extend(tokenized_w)

            orig_to_tok_map[i] = len(bert_tokens) - 1

        tok_to_orig_map = {}

        bert_tokens.append("[SEP]")
        tok_to_orig_map = get_tok_to_orig_map(orig_to_tok_map, len(original_sentence), len(bert_tokens))        
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(bert_tokens)
        tokens_tensor = torch.tensor([indexed_tokens]).to(self.device)

        return (bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor)
    
    
    def forward(self, x):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        return states
    
    def forward_with_loss_calculation2(self, bert_tokens, x, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        states = states / (torch.norm(states, dim = 1, keepdim = True)+1e-7)
        
        arg1_sent1, arg2_sent1 = range_sent1
        arg1_sent2, arg2_sent2 = range_sent2
        
        #print(range_sent1)
        #print(arg1_sent1, arg2_sent1)
        #print(range_sent2)
        #print(arg1_sent2, arg2_sent2)
        #print(states[arg1_sent1[0]:arg1_sent1[1]].shape)
        #print("sent1 args (gold):", bert_tokens[arg1_sent1[0]+3:arg1_sent1[1]], bert_tokens[arg2_sent1[0]+3:arg2_sent1[1]])
        #print("sent2 args (gold):", bert_tokens[arg1_sent2[0]:arg1_sent2[1]], bert_tokens[arg2_sent2[0]:arg2_sent2[1]])
        #print("------------------------------------------------")
        
        sent1_arg1_vec, sent1_arg2_vec = states[arg1_sent1[0]+3:arg1_sent1[1]].mean(dim=0), states[arg2_sent1[0]+3:arg2_sent1[1]].mean(dim=0)
        sent2_arg1_vec, sent2_arg2_vec = states[arg1_sent2[0]:arg1_sent2[1]].mean(dim=0), states[arg2_sent2[0]:arg2_sent2[1]].mean(dim=0)
        #sent1_arg1_vec, sent1_arg2_vec = states[arg1_sent1[0]:arg1_sent1[1]].mean(dim=0), states[arg2_sent1[0]:arg2_sent1[1]].mean(dim=0)
        #sent2_arg1_vec, sent2_arg2_vec = states[arg1_sent2[0]:arg1_sent2[1]].mean(dim=0), states[arg2_sent2[0]:arg2_sent2[1]].mean(dim=0)

        
        dists_arg1 = ((self.linear_arg1_1(states)-self.linear_arg1_2(sent1_arg1_vec))**2).sum(dim = 1).detach().cpu().numpy()
        dists_arg2 = ((self.linear_arg2_1(states)-self.linear_arg2_2(sent1_arg2_vec)**2)).sum(dim = 1).detach().cpu().numpy()       
        #states_projected = self.linear_arg1_1(states)
        #arg1_sent1_projected = self.linear_arg1_2(sent1_arg1_vec)
        #arg2_sent1_projected = self.linear_arg2_2(sent1_arg2_vec)
        #dists_arg1 = cosine_distance(arg1_sent1_projected, states_projected)
        #dists_arg2 = cosine_distance(arg2_sent1_projected, states_projected)
        
        
        mask_gold_arg1 = np.zeros_like(dists_arg1)
        mask_gold_arg2 = np.zeros_like(dists_arg2)
        mask_gold_arg1[arg1_sent1[0]:arg1_sent1[1]] = 1e6
        mask_gold_arg1[arg1_sent2[0]:arg1_sent2[1]] = 1e6
        mask_gold_arg2[arg2_sent1[0]:arg2_sent1[1]] = 1e6
        mask_gold_arg2[arg2_sent2[0]:arg2_sent2[1]] = 1e6
        mask_gold_arg2[:l_tokens] = 1e6
        mask_gold_arg1[:l_tokens] = 1e6

        
        idx_arg1 = np.argsort(dists_arg1+mask_gold_arg1)
        idx_arg2 = np.argsort(dists_arg2+mask_gold_arg2)
        k = 0# if np.random.random() < 0.5 else random.choice(range(10))
        
        alpha = 1
        dist_arg1_argmax = alpha*(self.linear_arg1_2(sent1_arg1_vec) - self.linear_arg1_1(states[idx_arg1[k]])).norm()#**2
        dist_arg2_argmax = alpha*(self.linear_arg2_2(sent1_arg2_vec) - self.linear_arg2_1(states[idx_arg2[k]])).norm()#**2
        dist_arg1_gold = alpha*(self.linear_arg1_2(sent1_arg1_vec) - self.linear_arg1_1(sent2_arg1_vec)).norm()#**2
        dist_arg2_gold = alpha*(self.linear_arg2_2(sent1_arg2_vec) - self.linear_arg2_1(sent2_arg2_vec)).norm()#**2
        
        #print("arg1-argmax, arg1-gold: {}, {}".format(dist_arg1_argmax, dist_arg1_gold))
        #print("arg2-argmax, arg2-gold: {}, {}".format(dist_arg2_argmax, dist_arg2_gold))
        
        # softmax triplet
        
        z = torch.max(dist_arg1_argmax, dist_arg1_gold)
        temp = 2
        pos_arg1 = torch.exp((dist_arg1_gold - z)/temp)
        neg_arg1 = torch.exp((dist_arg1_argmax - z)/temp)
        loss_arg1 = (pos_arg1 / (pos_arg1 + neg_arg1))**2

        z = torch.max(dist_arg2_argmax, dist_arg2_gold)
        pos_arg2 = torch.exp((dist_arg2_gold - z)/temp)
        neg_arg2 = torch.exp((dist_arg2_argmax - z)/temp)
        loss_arg2 = (pos_arg2 / (pos_arg2 + neg_arg2))**2

        alpha = 0.005
        loss_arg1 = torch.max(torch.zeros(1).to(self.device), dist_arg1_gold - dist_arg1_argmax + alpha)
        loss_arg2 = torch.max(torch.zeros(1).to(self.device), dist_arg2_gold - dist_arg2_argmax + alpha)
        #print(dist_arg1_gold.detach().cpu().numpy().item(), dist_arg1_argmax.detach().cpu().numpy().item(), dist_arg2_gold.detach().cpu().numpy().item(), dist_arg2_argmax.detach().cpu().numpy().item())
        global count
        global total
        if pos_arg1 < neg_arg1 and pos_arg2 < neg_arg2:
            count += 1
        total += 1                                                      
        # check
        
        if (not np.isnan(loss_arg2.detach().cpu().numpy().item())) and (not np.isnan(loss_arg1.detach().cpu().numpy().item())):
            
            loss = loss_arg1 + loss_arg2
        else:
            
            if not np.isnan(loss_arg2.detach().cpu().numpy().item()):
                loss = loss_arg2
            else:
                loss = loss_arg1
            
        if loss.detach().cpu().numpy().item() > 1e4:
            print("EEEEE")
            print(dist_arg1_gold, dist_arg1_argmax, dist_arg2_gold, dist_arg2_argmax)
            print("================================")
            print(dists_arg1)
            print("--------------")
            print(dists_arg2)
            return
        return loss, np.argsort(dists_arg1+mask_gold_arg1)
    
    def forward_with_loss_calculation(self, x, sent1_idx, sent2_idx, orig_to_tok_map, tok_to_orig_map, l, l_tokens):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        #states = states/torch.norm(states, dim = 1, keepdim = True)
        
        sent1_idx_arg1 = sent1_idx[0].detach().cpu().numpy().item()
        sent1_idx_arg2 = sent1_idx[1].detach().cpu().numpy().item()
        gold_arg1_sent1 = orig_to_tok_map[sent1_idx_arg1]
        gold_arg2_sent1 = orig_to_tok_map[sent1_idx_arg2]
        
        sent1_arg1_vec, sent1_arg2_vec = states[gold_arg1_sent1], states[gold_arg2_sent1]
        
        states_sent2 = states[orig_to_tok_map[l.detach().cpu().numpy().item()]:]  
        
        sims_arg1 = sent1_arg1_vec @ states.T
        sims_arg2 = sent1_arg2_vec @ states.T
        
        sent2_idx_arg1 = sent2_idx[0].detach().cpu().numpy().item()
        sent2_idx_arg2 = sent2_idx[1].detach().cpu().numpy().item()
        gold_arg1_sent2, gold_arg2_sent2 = orig_to_tok_map[sent2_idx_arg1], orig_to_tok_map[sent2_idx_arg2]

        dists_arg1 = ((self.linear_arg1(states-sent1_arg1_vec))**2).sum(dim = 1).detach().cpu().numpy()
        dists_arg2 = ((self.linear_arg2(states-sent1_arg2_vec)**2)).sum(dim = 1).detach().cpu().numpy()
               
        mask_gold_arg1 = np.zeros_like(dists_arg1)
        mask_gold_arg2 = np.zeros_like(dists_arg2)
        mask_gold_arg1[gold_arg1_sent2] = 1e6
        mask_gold_arg2[gold_arg2_sent2] = 1e6
        mask_gold_arg2[:l_tokens] = 1e6
        mask_gold_arg1[:l_tokens] = 1e6
        
        if np.random.random() < 0.0*1e-1:
            print(dists_arg1+mask_gold_arg2)
            print("=========================")
        
        idx_arg1 = np.argsort(dists_arg1+mask_gold_arg1)
        idx_arg2 = np.argsort(dists_arg2+mask_gold_arg2)
        
        k = 0 #random.choice(range(0,7))
        #idx_arg1 = [orig_to_tok_map[i] for i in orig_to_tok_map.keys() if i != sent2_idx_arg1 and i > l]
        #idx_arg2 = [orig_to_tok_map[i] for i in orig_to_tok_map.keys() if i != sent2_idx_arg2 and i > l]
        #random.shuffle(idx_arg1)
        #random.shuffle(idx_arg2)
        
        dist_arg1_argmax = (self.linear_arg1(sent1_arg1_vec - states[idx_arg1[k]])).norm()#**2
        dist_arg2_argmax = (self.linear_arg2(sent1_arg2_vec - states[idx_arg2[k]])).norm()#**2
        dist_arg1_gold = (self.linear_arg1(sent1_arg1_vec - states[gold_arg1_sent2])).norm()#**2
        dist_arg2_gold = (self.linear_arg2(sent1_arg2_vec - states[gold_arg2_sent2])).norm()#**2
        
        
        #alpha = 0.01
        #loss_arg1 = torch.max(torch.zeros(1).to(self.device), -sim_arg1_gold + most_sim_arg1 + alpha)
        #loss_arg2 = torch.max(torch.zeros(1).to(self.device), -sim_arg2_gold + most_sim_arg2 + alpha)
        alpha = 0.05
        loss_arg1 = torch.max(torch.zeros(1).to(self.device), dist_arg1_gold - dist_arg1_argmax + alpha)
        loss_arg2 = torch.max(torch.zeros(1).to(self.device), dist_arg2_gold - dist_arg2_argmax + alpha)

        # softmax triplet
        
        z = torch.max(dist_arg1_argmax, dist_arg1_gold)
        temp = 1
        pos_arg1 = torch.exp((dist_arg1_gold - z)/temp)
        neg_arg1 = torch.exp((dist_arg1_argmax - z)/temp)
        loss_arg1 = (pos_arg1 / (pos_arg1 + neg_arg1))**2

        z = torch.max(dist_arg2_argmax, dist_arg2_gold)
        pos_arg2 = torch.exp((dist_arg2_gold - z)/temp)
        neg_arg2 = torch.exp((dist_arg2_argmax - z)/temp)
        loss_arg2 = (pos_arg2 / (pos_arg2 + neg_arg2))**2
                                                               
        # check
        global count
        
        if tok_to_orig_map[idx_arg1[0]] == sent2_idx_arg1:
            print("yay",)
            count += 1
            loss_arg1 = torch.zeros(1).to(self.device).squeeze()
            
        if tok_to_orig_map[idx_arg2[0]] == sent2_idx_arg2:
            print("yay2")
            count += 1
            loss_arg2 = torch.zeros(1).to(self.device).squeeze()                                                      
            
        loss = loss_arg1 + loss_arg2
        return loss, np.argsort(dists_arg1+mask_gold_arg1)
    
    def training_step(self, batch, batch_nb):
        
        sents_concat, sent1_idx, sent2_idx, l = batch
        sent1_idx = [x.detach().cpu().numpy().item() for x in sent1_idx]
        sent2_idx = [x.detach().cpu().numpy().item() for x in sent2_idx]
        
        #print(sents_concat)
        #print("---------------------------------------------")
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat[0].split(" "))        
        l_tokens = len(bert_tokens[:orig_to_tok_map[l.detach().cpu().numpy().item()-1]]) 
        sent1_range_arg1 = get_entity_range(sent1_idx[0], orig_to_tok_map)
        sent1_range_arg2 = get_entity_range(sent1_idx[1], orig_to_tok_map)
        sent2_range_arg1 = get_entity_range(sent2_idx[0], orig_to_tok_map)
        sent2_range_arg2 = get_entity_range(sent2_idx[1], orig_to_tok_map)
        range_sent1 = (sent1_range_arg1,sent1_range_arg2)
        range_sent2 = (sent2_range_arg1,sent2_range_arg2)
        
        loss, argsort = self.forward_with_loss_calculation2(bert_tokens, tokens_tensor, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens)
        
        if np.isnan(loss.detach().cpu().numpy().item()) or loss.detach().cpu().numpy().item() > 1e4:
            print("ERRROR!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(sents_concat, range_sent1, range_sent2, sent1_idx, sent2_idx)
            return {"loss": loss*0}
        #loss, argsort = self.forward_with_loss_calculation(tokens_tensor, sent1_idx, sent2_idx, orig_to_tok_map,tok_to_orig_map, l, l_tokens)
        #print(sents_concat)
        #print(argsort[:5])
        #print("preds", bert_tokens[argsort[0]], bert_tokens[argsort[1]], bert_tokens[argsort[2]])
        #print("gold", bert_tokens[orig_to_tok_map[sent2_idx[0]]])
        #if np.random.random() < 1e-3:
        if total%1000 == 0:
            print("count", count/total)
        #print("==================================")
        return {'loss': loss}
    
    def validation_step(self, batch, batch_nb):

        sents_concat, sent1_idx, sent2_idx, l = batch
        #print(sent1_idx, type(sent1_idx))
        
        sent1_idx = [x.detach().cpu().numpy().item() for x in sent1_idx]
        sent2_idx = [x.detach().cpu().numpy().item() for x in sent2_idx]
        
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat[0].split(" "))
        l_tokens = len(bert_tokens[:orig_to_tok_map[l.detach().cpu().numpy().item()-1]]) 
        sent1_range_arg1 = get_entity_range(sent1_idx[0], orig_to_tok_map)
        sent1_range_arg2 = get_entity_range(sent1_idx[1], orig_to_tok_map)
        sent2_range_arg1 = get_entity_range(sent2_idx[0], orig_to_tok_map)
        sent2_range_arg2 = get_entity_range(sent2_idx[1], orig_to_tok_map)
        range_sent1 = (sent1_range_arg1,sent1_range_arg2)
        range_sent2 = (sent2_range_arg1,sent2_range_arg2)
        loss, argsort = self.forward_with_loss_calculation2(bert_tokens, tokens_tensor, range_sent1, range_sent2, orig_to_tok_map, l, l_tokens)
        #loss, argsort = self.forward_with_loss_calculation(tokens_tensor, sent1_idx, sent2_idx, orig_to_tok_map, tok_to_orig_map, l, l_tokens)

        return {'val_loss': loss}
    
    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        #return torch.optim.RMSprop(self.parameters())
        #return torch.optim.ASGD(self.parameters())
        return torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.75)
        #return torch.optim.Adam(self.parameters())
    
    @pl.data_loader
    def train_dataloader(self):
        return self.train_gen

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        # can also return a list of val dataloaders
        return self.dev_gen
    

def get_entity_range(index_orig, orig_to_tok_map):
    
    m = min(orig_to_tok_map.keys())
    if orig_to_tok_map[index_orig] == 1: return (1,2)
    if index_orig == 0: return (1, orig_to_tok_map[index_orig] + 1)
    
    before = index_orig - 1
    tok_range = (orig_to_tok_map[before] + 1, orig_to_tok_map[index_orig] + 1)
    return tok_range


def get_tok_to_orig_map(orig_to_tok_map, num_words, num_tokens):
    
    ranges = [get_entity_range(i, orig_to_tok_map) for i in range(num_words)]
    tok_to_orig_map = {}
    for i in range(num_words):
        min,max = ranges[i]
        for tok in range(min,max):
            tok_to_orig_map[tok] = i
    
    for tok in range(num_tokens):
        if tok not in tok_to_orig_map:
            tok_to_orig_map[tok] = num_words -1
    
    return tok_to_orig_map
        

def get_prediction(sent1, sent2, model):
    
    l = len(sent1.split(" ")) + 1 
    sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
    
    sents_concat = sent1 + " ***** " + sent2
    sent1_arg1 = [i for i,w in enumerate(sent1.split(" ")) if "ARG1:" in w][0]
    sent1_arg2 = [i for i,w in enumerate(sent1.split(" ")) if "ARG2:" in w][0]
    words = sents_concat.split(" ")
    
    bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sents_concat.split(" "))
    range_arg1 = get_entity_range(sent1_arg1, orig_to_tok_map)
    range_arg2 = get_entity_range(sent1_arg2, orig_to_tok_map)
    
    l_tokens = len(bert_tokens[:orig_to_tok_map[l-1]])    
    outputs = model.model(tokens_tensor)
    states = outputs[0][0] #[seq_len, 768]

    states = states/torch.norm(states, dim = 1, keepdim = True)
    
    sent1_arg1_vec, sent1_arg2_vec = states[range_arg1[0]:range_arg1[1]].mean(dim=0), states[range_arg2[0]:range_arg2[1]].mean(dim=0)
    
    print("GOLD ARG1, ARG2:", bert_tokens[orig_to_tok_map[sent1_arg1]], bert_tokens[orig_to_tok_map[sent1_arg2]])
    sims_arg1 = (model.linear_arg1_1(states)-model.linear_arg1_2(sent1_arg1_vec)).norm(dim=1)
    sims_arg2 = (model.linear_arg2_1(states)-model.linear_arg2_2(sent1_arg2_vec)).norm(dim=1)

    mask_gold_arg1 = torch.zeros_like(sims_arg1).to(model.device)
    mask_gold_arg2 = torch.zeros_like(sims_arg2).to(model.device)
    #mask_gold_arg1[range_arg1[0]:range_arg1[1]] = 1e6
    #mask_gold_arg2[range_arg2[0]:range_arg2[1]] = 1e6
    mask_gold_arg2[:l_tokens] = 1e6
    mask_gold_arg1[:l_tokens] = 1e6
        
    idx_arg1 = torch.argsort(sims_arg1+mask_gold_arg1).detach().cpu().numpy()
    idx_arg2 = torch.argsort(sims_arg2+mask_gold_arg2).detach().cpu().numpy()
    
    print("all arg1 preds", [words[tok_to_orig_map[idx_arg1[i]]] for i in range(len(idx_arg1))][:7])
    print("---------------------------------------------------")
    print("all arg2 preds", [words[tok_to_orig_map[idx_arg2[i]]] for i in range(len(idx_arg2))][:7])
    print("---------------------------------------------------")
    
    #print(bert_tokens[idx_arg1[-4]], bert_tokens[idx_arg2[-4]])
    
    return
    
    most_sim_arg1, most_sim_arg2 = sims_arg1[idx_arg1[-2]], sims_arg2[idx_arg2[-2]]
    ind_arg1, ind_arg2 = idx_arg1[-2].detach().cpu().numpy().item(), idx_arg2[-2].detach().cpu().numpy().item()
    ind_arg1, ind_arg2 = ind_arg1 + l_tokens, ind_arg2 + l_tokens
    
    return sents_concat, tok_to_orig_map[ind_arg1], tok_to_orig_map[ind_arg2], bert_tokens[ind_arg1], bert_tokens[ind_arg2]

In [79]:
class Dataset(torch.utils.data.Dataset):
    """Simple torch dataset class"""

    def __init__(self, data: List[Dict], device = "cpu"):

        self.data = data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        with torch.no_grad():
            
            d = self.data[index]
            sent1, sent2 = d["first"], d["second"]
            sent1_arg1, sent1_arg2 = d["first_arg1"][0], d["first_arg2"][0]
            sent2_arg1, sent2_arg2 = d["second_arg1"][0], d["second_arg2"][0]
            
            l = len(sent1.split(" ")) + 1 
            sent2_arg1, sent2_arg2 = sent2_arg1 + l, sent2_arg2 + l

            sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
            sents_concat = sent1 + " ***** " + sent2 #sents_concat.split(" ")[l] is the first token in the 2nd sent
            
            return sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l

In [80]:
l = int(0.8 * len(data))
train_dataset, dev_dataset = Dataset(data[:l], "cpu"), Dataset(data[l:], "cpu")

In [45]:
sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l = train_dataset[5]

In [81]:
del model

In [82]:
model = BertModel(train_dataset, dev_dataset, 1, "cuda", "train")

In [ ]:
sent = "it is very important."
bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sent.split(" "))
indexed_tokens = model.tokenizer.convert_tokens_to_ids(bert_tokens)
tokens_tensor = torch.tensor([indexed_tokens])

In [ ]:
bert_tokens

In [ ]:
orig_to_tok_map

In [ ]:
tok_to_orig_map

In [83]:
#model.to("cpu")
#model.device = "cpu"
trainer = Trainer(max_nb_epochs=50,min_nb_epochs=1, gpus = 1)
trainer.fit(model)

2020-11-01 11:57:02.033 INFO    lightning: GPU available: True, used: True
2020-11-01 11:57:02.034 INFO    lightning: CUDA_VISIBLE_DEVICES: [0]
2020-11-01 11:57:02.140 INFO    lightning: 
    | Name                                              | Type              | Params
------------------------------------------------------------------------------------
0   | model                                             | BertModel         | 109 M 
1   | model.embeddings                                  | BertEmbeddings    | 24 M  
2   | model.embeddings.word_embeddings                  | Embedding         | 23 M  
3   | model.embeddings.position_embeddings              | Embedding         | 393 K 
4   | model.embeddings.token_type_embeddings            | Embedding         | 1 K   
5   | model.embeddings.LayerNorm                        | LayerNorm         | 1 K   
6   | model.embeddings.dropout                          | Dropout           | 0     
7   | model.encoder                            

Loss is 0.05623554065823555


count 0.394
count 0.472
count 0.512
count 0.53075
count 0.5502


Loss is 0.006185966543853283
count 0.584
count 0.601125
count 0.6158888888888889
count 0.6263
count 0.6368181818181818


Loss is 0.0044884891249239445
count 0.6524615384615384
count 0.6622142857142858
count 0.6709333333333334
count 0.678125
count 0.6859411764705883


...

Loss is 0.004559141118079424
count 0.8687716535433071
count 0.869609375
count 0.8704108527131783
count 0.8711923076923077
count 0.8719541984732825


Loss is 0.004539175424724817
count 0.8719473684210526
count 0.8726641791044776
count 0.8733851851851852
count 0.8741029411764706
count 0.8747664233576642


Loss is 0.00460453238338232
count 0.8746043165467626
count 0.8753071428571428
count 0.8760567375886524
count 0.876669014084507
count 0.8773216783216783


2020-11-01 12:56:20.410 INFO    lightning: Detected KeyboardInterrupt, attempting graceful shutdown...


1

## model.model.layers

In [ ]:
del model

In [84]:
k = random.choice(range(100))

sent1 = "The use of 5 % ARG1:dextrose and sodium bicarbonate is an effective initial treatment for ARG2:hyperkalemia ."
sent2 = "Cyclosporine is an immunosuppressant and is used to avoid organ transplant rejection ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())

print("===================================================")
    
    
sent1 = "Suppose that a fraction ARG1:Q of all infected individuals is completely isolated and does not transmit the disease to ARG2:anyone ."
sent2 = "Although the J class is isolated to a negative-pressure room , a few of its members could transmit the virus , by accident , to other people such as healthcare workers ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "Moreover , the ARG1:pathogen also spreads to humans through direct ARG2:contact with infected poultry and contaminated surfaces [ 5 ] ."
sent2 = "The COVID-19 disease can spread in a population through infected symptomatic/asymptomatic individuals who come in contact directly/indirectly [ 4 ] ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "The low second trimester plasma MBL ARG1:level is not a risk factor for the development of ARG2:preterm birth ."
sent2 = "Nevertheless , even 5 - 10 years intervals from the previous gestation do not increase the risk for PE ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "However , ARG1:glucose blood levels were reduced and insulin sensitivity was increased in ARG2:MTX2 and MTX4 ."
sent2 = "Both blood glucose and lipid profiles were reduced in the allogenic group [ 9 ] ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "It seems likely that the prevention and treatment of ARG1:AEx of IPF must focus on both ARG2:disease-specific ( e.g. , anti-fi brotic therapies ) and non-disease-specific ( e.g. , vaccination , prevention of stress ) areas ."
sent2 = "Thus , the clinical management of SARS should consider not only antiviral but anti-inflammatory strategies as well ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
    
print("===================================================")
    
    
sent1 = "The results indicate that supplementation of fish with ARG1:rosemary could improve the haematological and immunological properties and increase the survival rate after challenge with ARG2:S. iniae ."
sent2 = "Recent studies have demonstrated that prolonged treatment with EP can ameliorate experimental ulcerative colitis and slow multiple tumor growth ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    
    
sent1 = "In 2013 , an outbreak of ARG1:MERS-CoV occurred in the ARG2:Middle East , including Jordan ( 10 , 11 ) ."
sent2 = "An outbreak of severe acute respiratory syndrome ( SARS ) was detected in Singapore at the beginning of March 2003 ."

with torch.no_grad():
    print(sent1)
    print("-------------------------------")
    print(sent2)
    print("-------------------------------")
    get_prediction(sent1, sent2, model.eval())
    
    
print("===================================================")
    

for i in range(10):

    sent1 = data[i]["first"]
    sent2 = data[i]["second"].replace("ARG1:", "").replace("ARG2:","")
    with torch.no_grad():
        print(sent1)
        print("-------------------------------")
        print(sent2)
        print("-------------------------------")
        get_prediction(sent1, sent2, model.eval())

The use of 5 % ARG1:dextrose and sodium bicarbonate is an effective initial treatment for ARG2:hyperkalemia .
-------------------------------
Cyclosporine is an immunosuppressant and is used to avoid organ transplant rejection .
-------------------------------
GOLD ARG1, ARG2: ##rose ##emia
all arg1 preds ['an', 'Cyclosporine', 'is', 'transplant', 'organ', 'Cyclosporine', 'is']
---------------------------------------------------
all arg2 preds ['organ', 'avoid', 'Cyclosporine', '.', '*****', 'an', 'Cyclosporine']
---------------------------------------------------
Suppose that a fraction ARG1:Q of all infected individuals is completely isolated and does not transmit the disease to ARG2:anyone .
-------------------------------
Although the J class is isolated to a negative-pressure room , a few of its members could transmit the virus , by accident , to other people such as healthcare workers .
-------------------------------
GOLD ARG1, ARG2: q anyone
all arg1 preds ['the', 'healthcare',

all arg1 preds ['This', 'infants', 'could', '.', 'age', 'neuraminidase', 'the']
---------------------------------------------------
all arg2 preds ['This', '.', 'children', '*****', 'no', 'infants', '1']
---------------------------------------------------


In [ ]:
del model

In [ ]:
torch.randn(1)*0

In [ ]:
data[0]